### Table of contents
[Predefined functions](#funcs)

[Parameters, filenames and directories](#params)

[Reading maps and catalogs](#read)

[SpecZ outlier filtering](#specZ)

[PhotoZ outlier filtering](#photoZ)

[ActivMap filtering](#activMap)

In [ ]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from func_maps import readMaps
from func_stat import *
